In [1]:
# 0. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import numpy as np
import pandas as pd
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from scipy.stats.mstats import winsorize
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Load Dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/diabetes_012_health_indicators_BRFSS2015.csv')
X = data.drop('Diabetes_012', axis=1)

# 4. Preprocessing (Sesuai Model Default)
X['BMI'] = winsorize(X['BMI'], limits=[0.005, 0.005])
X['MentHlth'] = np.where(X['MentHlth'] > 30, 30, X['MentHlth'])
X['PhysHlth'] = np.where(X['PhysHlth'] > 30, 30, X['PhysHlth'])

robust_features = ['BMI', 'MentHlth', 'PhysHlth']
minmax_features = ['Age', 'Education', 'Income', 'GenHlth']

scaler_robust = RobustScaler()
scaler_minmax = MinMaxScaler()

X[robust_features] = scaler_robust.fit_transform(X[robust_features])
X[minmax_features] = scaler_minmax.fit_transform(X[minmax_features])

# 5. Split Data untuk AE (tidak butuh label y)
X_train_ae, X_test_ae = train_test_split(X, test_size=0.2, random_state=SEED)

# 6. Build Autoencoder AE B2 dengan LeakyReLU
input_dim = X.shape[1]
input_layer = Input(shape=(input_dim,))

# Encoder
x = Dense(16)(input_layer)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(12)(x)
x = LeakyReLU(alpha=0.1)(x)

# Latent space
latent = Dense(10)(x)
latent = LeakyReLU(alpha=0.1)(latent)

# Decoder
x = Dense(12)(latent)
x = LeakyReLU(alpha=0.1)(x)
x = Dense(16)(x)
x = LeakyReLU(alpha=0.1)(x)

# Output layer
output_layer = Dense(input_dim, activation='linear')(x)

# Autoencoder model
autoencoder = Model(inputs=input_layer, outputs=output_layer)
autoencoder.compile(optimizer='adam', loss='mse')

# 7. Train Autoencoder
history = autoencoder.fit(
    X_train_ae, X_train_ae,
    validation_data=(X_test_ae, X_test_ae),
    epochs=100,
    batch_size=50,
    verbose=1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
)

# 8. Simpan Model
save_dir = '/content/drive/MyDrive/Colab Notebooks/Skripsi/Model/Model Autoencoders'
os.makedirs(save_dir, exist_ok=True)
# autoencoder.save(f'{save_dir}/aeb2_model_leakyrelu.h5')

# 9. Ekstrak Encoder dari Autoencoder
encoder = Model(inputs=input_layer, outputs=latent)

# 10. Transformasi seluruh data X menjadi bentuk laten
X_latent = encoder.predict(X)

# 11. Buat DataFrame hasil reduksi dimensi
latent_df = pd.DataFrame(X_latent, columns=[f'Latent_{i+1}' for i in range(X_latent.shape[1])])

# 12. Tambahkan label (opsional)
latent_df['Diabetes_012'] = data['Diabetes_012']

# 13. Simpan ke direktori yang ditentukan
output_path = '/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/V2 Hasil Eksperimen/AE'
os.makedirs(output_path, exist_ok=True)
latent_df.to_csv(f'{output_path}/AEB2_dataset_latent.csv', index=False)

print("✅ Dataset latent berhasil disimpan.")


Mounted at /content/drive
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.2559 - val_loss: 0.0549
Epoch 2/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0529 - val_loss: 0.0462
Epoch 3/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0436 - val_loss: 0.0397
Epoch 4/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0391 - val_loss: 0.0381
Epoch 5/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0381 - val_loss: 0.0377
Epoch 6/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0377 - val_loss: 0.0375
Epoch 7/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 0.0375 - val_loss: 0.0372
Epoch 8/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0373 - val_loss: 0.0371
Epoch 9/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.0372 - val_loss: 0.0372
Epoch 10/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 0.0371 - val_loss: 0.0369
Epoch 11/100
4059/4059 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0370 - val_loss: 0.0368
Epoch 12/100
4059/4059 ━